<a href="https://colab.research.google.com/github/97Whitehat/Curso2020-2021/blob/master/Task07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Task 07: Querying RDF(s)**

In [1]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

     |████████████████████████████████| 235kB 2.7MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 


Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [3]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=Nb056d1f7d1da4a56b726dedc6ffd5668 (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [16]:
ns = Namespace("http://somewhere#")
vcard = Namespace("http://www.w3.org/2001/vcard-rdf/3.0#")
for s, p, o in g:
  print(s,p,o)

from rdflib.plugins.sparql import prepareQuery

query1 =prepareQuery('''
SELECT ?subcl
  WHERE {
    ?subcl rdfs:subClassOf ns:Person.
  }
  ''',
    initNs = {"rdfs":RDFS, "ns":ns}
)
print ("----------------------------------------")
for res in g.query(query1):
  print (res)


http://somewhere#JaneSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Researcher
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Datatype
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN jane Smith
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Datatype
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#string
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001

**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [20]:
for s, p, o in g.triples((None, RDF.type, ns.Person)):
  print(s)
for s, p, o in g.triples((None, RDFS.subClassOf, ns.Person)):
  print(s)  
query = prepareQuery('''
  SELECT ?individuals 
  WHERE {
    {
      ?individuals rdf:type ns:Person.
    }
    UNION
    {
      ?subcl rdfs:subClassOf ns:Person.
      ?indiv rdf:type ?subcl
    }
  }
  ''',
  initNs = {"rdfs":RDFS, "rdf":RDF, "ns":ns}
)
print("--------------------------")
for res in g.query(query):
  print(res)

http://somewhere#JohnSmith
http://somewhere#SaraJones
http://somewhere#Researcher
--------------------------
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#SaraJones'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [24]:
for s, p, o in g.triples((None, RDF.type, ns.Person)):
  for s1, p1, o1 in g.triples((s, None, None)):
    print(s1,p1,o1)

for s, p, o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s1, p1, o1 in g.triples((None, RDF.type, s)):
    for s2, p2, o2 in g.triples((s1, None, None)):
      print(s2,p2,o2)
  
query = prepareQuery('''
  SELECT ?indiv ?prop ?class
  WHERE {
    {
      ?indiv rdf:type ns:Person.
      ?indiv ?prop ?class
    }
    UNION
    {
      ?indiv rdf:type ?s.
      ?indiv ?prop ?class.
      ?s rdfs:subClassOf ns:Person
    }
  }
  ''',
  initNs = {"rdfs":RDFS, "rdf":RDF, "ns":ns}
)
print("---------------------------------------------")
for res in g.query(query):
    print(res)

http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Jane
http://somewhere#JaneSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Researcher
http://somewhere#JaneSmith http://somewhere#h